In [1]:
# from tensorflow.keras.mixed_precision import set_global_policy
# set_global_policy('mixed_float16')


In [2]:
import os
import numpy as np
import pydicom

def load_dicom_series(root_folder):
    """
    Recursively loads all DICOM files from nested folders and organizes them into patient-wise volumes.
    """
    patient_volumes = {}  # Dictionary to store volumes per patient

    for dirpath, _, filenames in os.walk(root_folder):  # Walk through all subdirectories
        dicom_files = sorted([f for f in filenames if f.endswith(".dcm")])  # Sort for correct order

        if dicom_files:  # If the folder contains DICOM files
            patient_id = dirpath.split("/")[-2]  # Extract patient ID from folder name
            slices = [pydicom.dcmread(os.path.join(dirpath, f)).pixel_array for f in dicom_files]
            
            volume = np.stack(slices, axis=-1)  # Stack slices to form 3D volume
            
            patient_volumes[patient_id] = volume

    return patient_volumes

# Example usage
root_folder = "PPMI"  # Adjust to your dataset's root directory
spect_volumes = load_dicom_series(root_folder)

# Print details
for patient, volume in spect_volumes.items():
    print(f"Patient: {patient}, Shape: {volume.shape}")  # Example output: (128, 128, 64)


Patient: 2012-11-15_14_58_44.0, Shape: (91, 109, 91, 1)
Patient: 2010-10-28_15_13_36.0, Shape: (91, 109, 91, 1)
Patient: 2013-11-20_15_31_47.0, Shape: (91, 109, 91, 1)
Patient: 2015-12-02_09_47_00.0, Shape: (120, 128, 128, 1)
Patient: 2011-12-08_14_33_19.0, Shape: (91, 109, 91, 1)
Patient: 2013-10-09_14_32_48.0, Shape: (91, 109, 91, 1)
Patient: 2013-04-03_14_44_24.0, Shape: (91, 109, 91, 1)
Patient: 2015-09-23_14_19_08.0, Shape: (91, 109, 91, 1)
Patient: 2011-09-08_14_14_00.0, Shape: (91, 109, 91, 1)
Patient: 2013-01-15_14_14_32.0, Shape: (91, 109, 91, 1)
Patient: 2011-08-04_14_19_01.0, Shape: (91, 109, 91, 1)
Patient: 2016-04-12_13_42_56.0, Shape: (91, 109, 91, 1)
Patient: 2014-03-25_13_25_39.0, Shape: (91, 109, 91, 1)
Patient: 2012-01-05_14_13_48.0, Shape: (91, 109, 91, 1)
Patient: 2013-04-09_13_51_04.0, Shape: (91, 109, 91, 1)
Patient: 2012-11-27_13_06_40.0, Shape: (91, 109, 91, 1)
Patient: 2012-12-04_13_30_55.0, Shape: (91, 109, 91, 1)
Patient: 2012-04-05_12_55_54.0, Shape: (91, 10

In [3]:
import cv2

def preprocess_volume(volume, target_size=(64, 64, 64)):
    """
    Normalizes and resizes the 3D volume.
    """
    # Normalize (Scale values between 0 and 1)
    volume = volume.astype(np.float32) / np.max(volume)

    # Resize each 2D slice correctly
    # Remove the extra channel dimension if present
    if volume.ndim == 4 and volume.shape[-1] == 1:
        volume = np.squeeze(volume, axis=-1)

    resized_slices = [cv2.resize(slice, target_size[:2]) for slice in volume[..., 0]]
    resized_volume = np.stack(resized_slices, axis=-1)  # Stack resized slices

    return np.expand_dims(resized_volume, axis=-1)  # Add channel dimension

# Example preprocessing

spect_volumes = dict(list(spect_volumes.items())[:100])
preprocessed_volumes = {pid: preprocess_volume(vol) for pid, vol in spect_volumes.items()}


In [4]:
import numpy as np
import cv2
import random

# Ensure all volumes have the same shape
volume_shape = (64, 64, 64, 1)  # Target shape

X_train = []
for vol in preprocessed_volumes.values():
    vol = np.squeeze(vol)  # Remove unnecessary singleton dimensions

    # Ensure the depth is correct
    num_slices = vol.shape[-1]
    if num_slices != 64:
        resized_slices = []
        for i in range(64):  # Always create exactly 64 slices
            idx = int(i * num_slices / 64)  # Sample indices proportionally
            resized_slices.append(cv2.resize(vol[:, :, idx], volume_shape[:2]))  
        vol = np.stack(resized_slices, axis=-1)  # Stack into depth

    vol = np.expand_dims(vol, axis=-1)  # Ensure (64, 64, 64, 1)
    X_train.append(vol)

X_train = np.array(X_train)  # Convert list to numpy array

# Generate random labels and one-hot encode them
y_train = np.array([random.randint(0, 1) for _ in range(len(X_train))])  # Random binary labels

print("X_train shape:", X_train.shape)  # Should be (N, 64, 64, 64, 1)
print("y_train shape:", y_train.shape)  # Should be (N,)


X_train shape: (100, 64, 64, 64, 1)
y_train shape: (100,)


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization

# Model architecture
model = Sequential([
    Conv3D(32, kernel_size=(3,3,3), activation='relu', input_shape=(64, 64, 64, 1)),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2,2,2)),

    Conv3D(64, kernel_size=(3,3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2,2,2)),

    Conv3D(128, kernel_size=(3,3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2,2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train
history = model.fit(X_train, y_train, epochs=10, batch_size=2, validation_split=0.2)


2025-04-02 20:39:40.057580: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743606580.066175  160217 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743606580.068560  160217 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743606580.075942  160217 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743606580.075957  160217 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743606580.075959  160217 computation_placer.cc:177] computation placer alr

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (None, 62, 62, 62, 32) │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 62, 62, 62, 32) │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 31, 31, 31, 32) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 29, 29, 29, 64) │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 29, 29, 29, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 14, 14, 14, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 12, 12, 12,     │       221,312 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 12,     │           512 │
│ (BatchNormalization)            │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 6, 6, 6, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 27648)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,539,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,825,857 (14.59 MB)

 Trainable params: 3,825,409 (14.59 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 30s 715ms/step - accuracy: 0.6852 - loss: 1.6150 - val_accuracy: 0.4000 - val_loss: 0.7385
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 702ms/step - accuracy: 0.4520 - loss: 1.6038 - val_accuracy: 0.6000 - val_loss: 0.6814
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 29s 717ms/step - accuracy: 0.5325 - loss: 1.8231 - val_accuracy: 0.4000 - val_loss: 0.7495
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 709ms/step - accuracy: 0.5860 - loss: 1.3157 - val_accuracy: 0.5000 - val_loss: 0.6985
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 703ms/step - accuracy: 0.4520 - loss: 1.2266 - val_accuracy: 0.4000 - val_loss: 1.1969
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 703ms/step - accuracy: 0.5435 - loss: 1.7440 - val_accuracy: 0.4000 - val_loss: 2.1907
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 701ms/step - accuracy: 0.5171 - loss: 1.8983 - val_accuracy: 0.6000 - val_loss: 0.6480
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 29s 716ms/step - accuracy: 0.4317 - loss: 1.5225 - val_accu

In [8]:
import pandas as pd

In [12]:
data = pd.read_csv('/home/antriksh/Desktop/DAV PROJECT/mil_gaya_4_02_2025.csv')
data

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I504421,51440,PD,M,70,SC,SPECT,Reconstructed DaTSCAN,Processed,4/10/2015,DCM,4/02/2025
1,I912277,51440,PD,M,72,V06,SPECT,Reconstructed DaTSCAN,Processed,5/11/2017,DCM,4/02/2025
2,I1333654,51440,PD,M,74,V10,SPECT,Reconstructed DaTSCAN,Processed,6/05/2019,DCM,4/02/2025
3,I678794,41488,PD,M,70,SC,SPECT,Reconstructed DaTSCAN,Processed,8/25/2015,DCM,4/02/2025
4,I388628,4140,Control,M,76,SC,SPECT,Reconstructed DaTSCAN,Processed,4/09/2013,DCM,4/02/2025
...,...,...,...,...,...,...,...,...,...,...,...,...
1769,I358111,3001,PD,M,66,V04,SPECT,Reconstructed DaTSCAN,Processed,4/06/2012,DCM,4/02/2025
1770,I449434,3001,PD,M,68,U02,SPECT,Reconstructed DaTSCAN,Processed,4/25/2014,DCM,4/02/2025
1771,I419651,3001,PD,M,67,V06,SPECT,Reconstructed DaTSCAN,Processed,5/03/2013,DCM,4/02/2025
1772,I504827,3001,PD,M,69,V10,SPECT,Reconstructed DaTSCAN,Processed,4/10/2015,DCM,4/02/2025


In [14]:
data["Group"].value_counts()

Group
PD         1424
Control     213
SWEDD       137
Name: count, dtype: int64